In [1]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv("./home-credit-default-risk/application_train.csv")
#test = pd.read_csv("./home-credit-default-risk/application_test.csv")

target = train['TARGET']
#features_name = test.columns

In [3]:
previous = pd.read_csv("./home-credit-default-risk/previous_application.csv")

In [4]:
previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
previous = previous.drop(columns=['DAYS_FIRST_DRAWING'])

In [6]:
previous['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
previous['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
previous['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
previous['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

In [7]:
def countNAN(df):
    d = {}
    for c in df:
        count = df[c].isnull().sum()
        if count != 0:
            d[c] = count / 1670214
    print(pd.DataFrame.from_dict(d,orient='index').sort_values(by = 0, ascending = False))

In [8]:
previous = previous.drop(['RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED',
                          'AMT_DOWN_PAYMENT', 'RATE_DOWN_PAYMENT'], axis = 1)
countNAN(previous)

                                      0
DAYS_TERMINATION           5.382412e-01
DAYS_LAST_DUE              5.294447e-01
NAME_TYPE_SUITE            4.911975e-01
DAYS_LAST_DUE_1ST_VERSION  4.591801e-01
DAYS_FIRST_DUE             4.273165e-01
NFLAG_INSURED_ON_APPROVAL  4.029813e-01
AMT_GOODS_PRICE            2.308177e-01
AMT_ANNUITY                2.228667e-01
CNT_PAYMENT                2.228637e-01
PRODUCT_COMBINATION        2.071591e-04
AMT_CREDIT                 5.987257e-07


In [9]:
previous.dtypes

SK_ID_PREV                       int64
SK_ID_CURR                       int64
NAME_CONTRACT_TYPE              object
AMT_ANNUITY                    float64
AMT_APPLICATION                float64
AMT_CREDIT                     float64
AMT_GOODS_PRICE                float64
WEEKDAY_APPR_PROCESS_START      object
HOUR_APPR_PROCESS_START          int64
FLAG_LAST_APPL_PER_CONTRACT     object
NFLAG_LAST_APPL_IN_DAY           int64
NAME_CASH_LOAN_PURPOSE          object
NAME_CONTRACT_STATUS            object
DAYS_DECISION                    int64
NAME_PAYMENT_TYPE               object
CODE_REJECT_REASON              object
NAME_TYPE_SUITE                 object
NAME_CLIENT_TYPE                object
NAME_GOODS_CATEGORY             object
NAME_PORTFOLIO                  object
NAME_PRODUCT_TYPE               object
CHANNEL_TYPE                    object
SELLERPLACE_AREA                 int64
NAME_SELLER_INDUSTRY            object
CNT_PAYMENT                    float64
NAME_YIELD_GROUP         

In [10]:
previous['AMT_ANNUITY'].describe()

count    1.297979e+06
mean     1.595512e+04
std      1.478214e+04
min      0.000000e+00
25%      6.321780e+03
50%      1.125000e+04
75%      2.065842e+04
max      4.180581e+05
Name: AMT_ANNUITY, dtype: float64

In [11]:
(previous['AMT_APPLICATION']>3000000).sum()

484

In [12]:
(previous['AMT_CREDIT'] > 3000000).sum()

628

In [13]:
(previous['AMT_GOODS_PRICE'] > 3000000).sum()

484

In [14]:
previous['AMT_APPLICATION'].median()

71046.0

In [15]:
previous.shape

(1670214, 32)

In [16]:
import project_preprocessing_helper
agg = project_preprocessing_helper.df_agg(previous, 'SK_ID_CURR', 'previous_app')

In [17]:
agg

,count,previous_app_AMT_ANNUITY_mean,previous_app_AMT_ANNUITY_max,previous_app_AMT_ANNUITY_min,previous_app_AMT_ANNUITY_sum,previous_app_AMT_APPLICATION_mean,previous_app_AMT_APPLICATION_max,previous_app_AMT_APPLICATION_min,previous_app_AMT_APPLICATION_sum,previous_app_AMT_CREDIT_mean,...,previous_app_PRODUCT_COMBINATION_Cash X-Sell: low_mean,previous_app_PRODUCT_COMBINATION_Cash X-Sell: middle_mean,previous_app_PRODUCT_COMBINATION_POS household with interest_mean,previous_app_PRODUCT_COMBINATION_POS household without interest_mean,previous_app_PRODUCT_COMBINATION_POS industry with interest_mean,previous_app_PRODUCT_COMBINATION_POS industry without interest_mean,previous_app_PRODUCT_COMBINATION_POS mobile with interest_mean,previous_app_PRODUCT_COMBINATION_POS mobile without interest_mean,previous_app_PRODUCT_COMBINATION_POS other with interest_mean,previous_app_PRODUCT_COMBINATION_POS others without interest_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,1,3951.000000,3951.000,3951.000,3951.000,24835.500,24835.5,24835.5,24835.5,23787.00,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.00,0.0,0.0,0.0
100002,1,9251.775000,9251.775,9251.775,9251.775,179055.000,179055.0,179055.0,179055.0,179055.00,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,1.0,0.0
100003,3,56553.990000,98356.995,6737.310,169661.970,435436.500,900000.0,68809.5,1306309.5,484191.00,...,0.333333,0.00,0.333333,0.0,0.333333,0.0,0.00,0.0,0.0,0.0
100004,1,5357.250000,5357.250,5357.250,5357.250,24282.000,24282.0,24282.0,24282.0,20106.00,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,1.0,0.0,0.0
100005,2,4813.200000,4813.200,4813.200,4813.200,22308.750,44617.5,0.0,44617.5,20076.75,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.50,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,1,6605.910000,6605.910,6605.910,6605.910,40455.000,40455.0,40455.0,40455.0,40455.00,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.00,0.0,0.0,0.0
456252,1,10074.465000,10074.465,10074.465,10074.465,57595.500,57595.5,57595.5,57595.5,56821.50,...,0.000000,0.00,0.000000,1.0,0.000000,0.0,0.00,0.0,0.0,0.0
456253,2,4770.405000,5567.715,3973.095,9540.810,24162.750,28912.5,19413.0,48325.5,20625.75,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.00,0.0,0.0,0.0


In [144]:
prev

SK_ID_PREV
